In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
interactome = [i.strip() for i in open('../../Body/1_Raw/HumanInteractome-union.tsv', 'r').readlines()]

In [3]:
interactome[0]

'ENSG00000000005\tENSG00000061656'

In [2]:
chaperome = pd.read_excel('../../Body/1_Raw/pre-Karras-chaperone-proteinr-interaction-profile.xlsx')

In [3]:
chaperome.head()

,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,WT_ELISA_score,Mut_ELISA_score,Both_expressed,WT_interaction_score,Mut_interaction_score,Differential_Z_score,Interact,Quality_control_factor_common,Quality_control_factor_official
0,Disease mutation,A4GALT,53947,53947_1850,NM_017436:c.287G>A,NP_059132:p.C96Y,0.51,0.79,1,1.398952,4.766989,3.105452,yes,BAG2,BAG2
1,Disease mutation,A4GALT,53947,53947_1852,NM_017436:c.299C>T,NP_059132:p.S100L,0.51,0.71,1,1.398952,5.029331,3.347341,yes,BAG2,BAG2
2,Disease mutation,A4GALT,53947,53947_1854,NM_017436:c.548T>A,NP_059132:p.M183K,0.51,0.48,1,1.398952,1.595533,0.181255,NaN,BAG2,BAG2
3,Disease mutation,A4GALT,53947,53947_1856,NM_017436:c.656C>T,NP_059132:p.A219V,0.51,0.75,1,1.398952,4.833744,3.167002,yes,BAG2,BAG2
4,Disease mutation,AAAS,8086,8086_5606,NM_015665:c.43C>A,NP_056480:p.Q15K,0.20,0.35,0,6.765101,9.823864,2.820292,NaN,BAG2,BAG2


In [6]:
chaperome.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16464 entries, 0 to 16463
Data columns (total 15 columns):
Category                           16464 non-null object
Symbol                             16464 non-null object
Entrez_Gene_ID                     16464 non-null int64
Allele_ID                          16464 non-null object
Mutation_RefSeq_NT                 16464 non-null object
Mutation_RefSeq_AA                 16464 non-null object
WT_ELISA_score                     16464 non-null float64
Mut_ELISA_score                    16464 non-null float64
Both_expressed                     16464 non-null int64
WT_interaction_score               16464 non-null float64
Mut_interaction_score              16464 non-null float64
Differential_Z_score               16464 non-null float64
Interact                           1169 non-null object
Quality_control_factor_common      16464 non-null object
Quality_control_factor_official    16464 non-null object
dtypes: float64(5), int64(2), obje

In [7]:
chaperome['Quality_control_factor_official'].unique()

array(['BAG2', 'STUB1', 'HSPA5', 'HSP90B1', 'HSPA8', 'HSP90AB1', 'PSMD2'],
      dtype=object)

In [8]:
chaperome['Quality_control_factor_common'].unique()

array(['BAG2', 'CHIP', 'GRP78', 'GRP94', 'HSC70', 'HSP90', 'PSMD2'],
      dtype=object)

Будем брать только взаимодействия с HSP90 и HSC70 (HSP90AB1 и HSPA8), так как остальные либо кошапероны, либо работают в ЭПР.

In [4]:
chaperome = chaperome[(chaperome['Quality_control_factor_common'] == 'HSP90') | (chaperome['Quality_control_factor_common'] == 'HSC70')]

In [10]:
print(chaperome.shape)
print(len(np.unique(chaperome['Entrez_Gene_ID'])))
chaperome.head()

(4804, 15)
997


,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,WT_ELISA_score,Mut_ELISA_score,Both_expressed,WT_interaction_score,Mut_interaction_score,Differential_Z_score,Interact,Quality_control_factor_common,Quality_control_factor_official
9328,Disease mutation,A4GALT,53947,53947_1850,NM_017436:c.287G>A,NP_059132:p.C96Y,0.63,0.58,1,0.602571,0.811199,0.426367,NaN,HSC70,HSPA8
9329,Disease mutation,A4GALT,53947,53947_1852,NM_017436:c.299C>T,NP_059132:p.S100L,0.63,0.41,1,0.602571,-0.130487,-1.498131,NaN,HSC70,HSPA8
9330,Disease mutation,A4GALT,53947,53947_1854,NM_017436:c.548T>A,NP_059132:p.M183K,0.63,0.58,1,0.602571,0.934306,0.677957,NaN,HSC70,HSPA8
9331,Disease mutation,A4GALT,53947,53947_1856,NM_017436:c.656C>T,NP_059132:p.A219V,0.63,0.60,1,0.602571,0.617262,0.030023,NaN,HSC70,HSPA8
9332,Disease mutation,AAAS,8086,8086_5606,NM_015665:c.43C>A,NP_056480:p.Q15K,0.40,0.38,1,3.401533,3.371882,-0.060597,NaN,HSC70,HSPA8


Из BioMart берем данные по соответсвию Entrez и Ensembl ID (https://m.ensembl.org/biomart/martview/3159cd65cf7ba8e1637d641e08d61854)

In [5]:
biomart_genes = pd.read_table('../../Body/1_Raw/biomart_human_ensembl_entrez_ids.txt')

In [6]:
biomart_genes = biomart_genes[['Gene stable ID', 'NCBI gene (formerly Entrezgene) ID']].dropna()

In [7]:
print(biomart_genes.shape)
biomart_genes.head()

(169019, 2)


,Gene stable ID,NCBI gene (formerly Entrezgene) ID
5,ENSG00000198888,4535.0
9,ENSG00000198763,4536.0
15,ENSG00000198804,4512.0
18,ENSG00000198712,4513.0
20,ENSG00000228253,4509.0


In [8]:
biomart_genes['NCBI gene (formerly Entrezgene) ID'] = biomart_genes['NCBI gene (formerly Entrezgene) ID'].astype(int)

In [9]:
len(set(chaperome['Entrez_Gene_ID']).intersection(set(biomart_genes['NCBI gene (formerly Entrezgene) ID'])))

996

In [10]:
biomart_genes[biomart_genes['NCBI gene (formerly Entrezgene) ID'].isin(chaperome['Entrez_Gene_ID'])]['Gene stable ID']

145       ENSG00000092330
150       ENSG00000092330
185       ENSG00000092330
190       ENSG00000092330
193       ENSG00000092330
               ...       
250804    ENSG00000284061
250805    ENSG00000284061
250806    ENSG00000284061
250807    ENSG00000284061
250808    ENSG00000284061
Name: Gene stable ID, Length: 11133, dtype: object

In [11]:
chaperome['EnsemlID'] = 0

In [12]:
for _id in np.unique(biomart_genes['NCBI gene (formerly Entrezgene) ID']):
    if _id in np.unique(chaperome['Entrez_Gene_ID']):
        chaperome.loc[chaperome[chaperome['Entrez_Gene_ID'] == _id].index,'EnsemlID'] = list(biomart_genes[biomart_genes['NCBI gene (formerly Entrezgene) ID'] == _id]['Gene stable ID'])[0]

In [13]:
len(chaperome['EnsemlID'].unique())

997

In [14]:
len(np.unique(chaperome['Entrez_Gene_ID']))

997

УРА

## Теперь посмотрим на пересечение
### Сначала для HSP90

In [22]:
hsp90_int = (chaperome[(chaperome['Quality_control_factor_common'] == 'HSP90')  & (chaperome['EnsemlID'] != 0) & (chaperome['WT_ELISA_score'] > 0.2)][['EnsemlID', 'WT_interaction_score']].drop_duplicates())
len(hsp90_int)

840

In [57]:
len(set(chaperome.Entrez_Gene_ID))

997

In [47]:
(chaperome[(chaperome['Quality_control_factor_common'] == 'HSC70')][['EnsemlID', 'WT_interaction_score']].drop_duplicates())

,EnsemlID,WT_interaction_score
9328,ENSG00000128274,0.602571
9332,ENSG00000094914,3.401533
9336,ENSG00000129673,0.764934
9337,ENSG00000129673,-1.042249
9338,ENSG00000183044,0.141185
...,...,...
11723,ENSG00000204644,1.925495
11725,ENSG00000155256,1.251179
11726,ENSG00000172466,1.847833
11727,ENSG00000147124,-0.687144


In [27]:
hsp90_int.to_csv('../../Body/2_Derived/hsp90_clients.Elisa_more_0.2.WT_interaction_score.csv', header = True, index=False)

In [30]:
hsp90_int[hsp90_int['WT_interaction_score'] >=3].shape

(146, 2)

#### Я фильтрую по WT_interaction_score >= 1, возможно, что порог должен быть больше.

In [22]:
for edge in interactome:
    if 'ENSG00000096384' in edge:
        print(edge)

ENSG00000096384	ENSG00000106993
ENSG00000096384	ENSG00000110172


In [23]:
for _id in chaperome[(chaperome['Quality_control_factor_common'] == 'HSP90') & (chaperome['WT_interaction_score'] > 0) & (chaperome['EnsemlID'] != 0)]['EnsemlID'].unique():
    if _id in ['ENSG00000106993','ENSG00000110172']:
        print(_id)

В референсном человеческом интерактоме всего 2 взаимодействия с HSP90, которых нет среди взаимодействий из статьи.

### Теперь посмотрим на HSC70

In [24]:
hsc70_int = (chaperome[(chaperome['Quality_control_factor_common'] == 'HSC70')  & (chaperome['EnsemlID'] != 0) & (chaperome['WT_ELISA_score'] > 0.2)][['EnsemlID', 'WT_interaction_score']].drop_duplicates())
len(hsc70_int)

817

In [28]:
hsc70_int.to_csv('../../Body/2_Derived/hsc70_clients.Elisa_more_0.2.WT_interaction_score.csv', header = True, index=False)

In [32]:
hsc70_int[hsc70_int['WT_interaction_score'] >=3].shape

(110, 2)

In [25]:
for edge in interactome:
    if 'ENSG00000109971' in edge:
        print(edge)

ENSG00000107262	ENSG00000109971
ENSG00000109971	ENSG00000112343
ENSG00000109971	ENSG00000123609
ENSG00000109971	ENSG00000133265
ENSG00000109971	ENSG00000151929
ENSG00000109971	ENSG00000156735


In [29]:
for _id in chaperome[(chaperome['Quality_control_factor_common'] == 'HSC70') & (chaperome['WT_interaction_score'] > 0) & (chaperome['EnsemlID'] != 0)]['EnsemlID'].unique():
    if _id in ['ENSG00000107262','ENSG00000112343', 'ENSG00000123609', 'ENSG00000133265', 'ENSG00000151929', 'ENSG00000156735']:
        print(_id)

А для HSC70 6, и также ни одного пересечения


In [33]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

In [38]:
len(intersection(list(hsc70_int[hsc70_int['WT_interaction_score'] >=3]['EnsemlID']), list(hsp90_int[hsp90_int['WT_interaction_score'] >=3]['EnsemlID'])))

70

Число общих клиентов для HSC70 и HSP90 - 211

## Сделаем таблицу со всеми клиентами и запишем ее в файл

In [39]:
hsp90_int = list(hsp90_int[hsp90_int['WT_interaction_score'] >=3]['EnsemlID'])
hsc70_int = list(hsc70_int[hsc70_int['WT_interaction_score'] >=3]['EnsemlID'])

In [40]:
clients_list = set(list(hsc70_int) + list(hsp90_int))

In [41]:
clients = pd.DataFrame({'clients' : list(clients_list), 'HSP90AB1': False, 'HSPA8' : False})

In [42]:
for client in clients_list:
    if client in hsp90_int:
        clients.loc[clients[clients['clients'] == client].index, 'HSP90AB1'] = True
    if client in hsc70_int:
        clients.loc[clients[clients['clients'] == client].index, 'HSPA8'] = True

In [45]:
clients.shape

(183, 3)

In [44]:
clients.to_csv('../../Body/2_Derived/Chaperone-Protein_Interaction_hsp90_hsc70_clients_int_score_more_three.csv', header = True, index=False)